# 22章　非構造化データ向けのニューラルネットワーク 
## レシピ22.1　画像クラス分類を行うニューラルネットワークの訓練 


In [ ]:
# ライブラリをロード
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# コンボリューショナルニューラルネットワークのアーキテクチャを定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(self.dropout1(x), 2)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(self.dropout2(x)))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# 実行するデバイスを設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの前処理を定義
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# MNISTデータセットをロード
train_dataset = datasets.MNIST('./data', train=True, download=True,
    transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

# データローダを作成
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
    shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
    shuffle=True)

# モデルと最適化器を初期化
model = Net().to(device)
optimizer = optim.Adam(model.parameters())

# torch 2.0の最適化器を用いてモデルをコンパイル
model = torch.compile(model)

# 訓練ループを定義
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = nn.functional.nll_loss(output, target)
    loss.backward()
    optimizer.step()

# テストループを定義
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        
        # 確率の対数が最大となるインデックスを取得
        test_loss += nn.functional.nll_loss(
            output, target, reduction='sum'
        ).item() # バッチロスに加算
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
test_loss /= len(test_loader.dataset)


## レシピ22.2　 テキストのクラス分類を行うニューラルネットワークの訓練 


In [ ]:
# ライブラリをロード
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 20 newsgroupデータセットをロード
cats = ['alt.atheism', 'sci.space']
newsgroups_data = fetch_20newsgroups(subset='all', shuffle=True,
    random_state=42, categories=cats)

# データセットを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(newsgroups_data.data,
    newsgroups_data.target, test_size=0.2, random_state=42)

# Bag-of-words手法を用いてテキストデータをベクトル化
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

# データをPyTorchのテンソルに変換
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# モデルを定義
class TextClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# モデルを作成し、ロス関数と最適化器を定義
model = TextClassifier(num_classes=len(cats))
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# torch 2.0の最適化器を用いてモデルをコンパイル
model = torch.compile(model)

# モデルを訓練
num_epochs = 1
batch_size = 10
num_batches = len(X_train) // batch_size
for epoch in range(num_epochs):
    total_loss = 0.0
    for i in range(num_batches):
        # 対象のバッチの入力とターゲットを用意
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        inputs = X_train[start_idx:end_idx]
        targets = y_train[start_idx:end_idx]

        # 最適化器の保持する勾配をゼロにリセット
        optimizer.zero_grad()

        # モデルにデータを順伝搬して損失を計算
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
    
        # 逆伝搬してパラメータを更新
        loss.backward()
        optimizer.step()

        # エポック全体のロスを更新
        total_loss += loss.item()

# このエポックでのテストセットに対する精度を計算
test_outputs = model(X_test)
test_predictions = torch.argmax(test_outputs, dim=1)
test_accuracy = accuracy_score(y_test, test_predictions)

# エポック番号、平均ロス、テスト精度を出力
print(f"Epoch: {epoch+1}, Loss: {total_loss/num_batches}, Test Accuracy:"
    f"{test_accuracy}")

In [ ]:
X_train.shape[1]

## レシピ22.3　 訓練済みモデルのファインチューニングによる画像クラス分類 


In [ ]:
# ライブラリをロード
import torch
from torchvision.transforms import(
    RandomResizedCrop, Compose, Normalize, ToTensor
    )
from transformers import Trainer, TrainingArguments, DefaultDataCollator
from transformers import ViTFeatureExtractor, ViTForImageClassification
from datasets import load_dataset, load_metric, Image

# 画像をRGBに変換するヘルパ関数を定義
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in
    examples["image"]]
    del examples["image"]
    return examples

# メトリクスを計算するヘルパ関数を定義
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids)

# fashion mnistデータセットをロード
dataset = load_dataset("fashion_mnist")

# VITモデルから画像プロセッサをロード
image_processor = ViTFeatureExtractor.from_pretrained(
    "google/vit-base-patch16-224-in21k"
)

# データセットから取り出したラベルを設定
labels = dataset['train'].features['label'].names

# 訓練済みモデルをロード
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

# コレータ、ノーマライザ、画像変換をロード。
collate_fn = DefaultDataCollator()
normalize = Normalize(mean=image_processor.image_mean,
    std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

# 変換に用いるデータセットをロード
dataset = dataset.with_transform(transforms)

# メトリクスとしてaccuracyを使用
metric = load_metric("accuracy")

# 訓練時の引数を設定
training_args = TrainingArguments(
    output_dir="fashion_mnist_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.01,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

# Trainerを作成
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
)

# モデルを訓練し、メトリクスを記録し、セーブする
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

## レシピ22.4　 訓練済みモデルのファインチューニングによるテキストクラス分類 


In [ ]:
# ライブラリをロード
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import (
    AutoModelForSequenceClassification, TrainingArguments, Trainer
    )
import evaluate
import numpy as np

# imdbデータセットをロード
imdb = load_dataset("imdb")

# トークナイザとコレータを作成
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# imdbデータセットをトークナイズ
tokenized_imdb = imdb.map(
    lambda example: tokenizer(
        example["text"], padding="max_length", truncation=True
    ),
    batched=True,
)

# accuracyをメトリクスとして使用
accuracy = evaluate.load("accuracy")

# メトリクスを計算するヘルパ関数を定義
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# インデックスとラベル間のマッピングを行う辞書を作成
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# 訓練済みモデルをロード
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

# 訓練引数を指定
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainerを作成
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# モデルを訓練
trainer.train()